In [35]:
# Imports

import json, yaml
import numpy as np
import pandas as pd
import datetime
import math
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

In [36]:
# Open connections

config_file = '/Users/annaartigas/ada/TarifaJusta/TJ_config.yml'
with open(config_file, 'r') as conf:
    config = yaml.load(conf)

db_core = config['db_core']
conn_core = create_engine(URL('postgresql', **db_core))
db_bi = config['db_bi']
conn_bi = create_engine(URL('mysql', **db_bi))

In [38]:
# Function to insert new data to database

def insertNewDataToDB(df, table, conn):

    # Find info already in DB
    already_in =  pd.read_sql(
        f"""
        SELECT * FROM {table}
        """, conn)
    # Find info not already in DB and insert
    df_notInDB = df[~df.isin(already_in)].dropna()
    if len(df_notInDB) > 0:
        df_notInDB.to_sql(name=table, con=conn, if_exists="append", index=False)
    
    return

In [39]:
# Retrieve phone calls data

CALLS =  pd.read_sql("""
SELECT date(calldate) as date , call_type, count(distinct puid) as n_calls, 
count(distinct source) as n_callers FROM MVC_ATC_DETAILS_FT A
left join MVC_ATC_SERVICE_FT B ON A.service = B.service 
where level_01 = '[Grupo de agentes]:Ventas'
and disposition = 'ANSWERED'
and duration > 60
and date(calldate) > (SELECT MAX(date) from OPE_CALLS)
group by date(calldate), call_type
    """, conn_bi)

insertNewDataToDB(CALLS, 'OPE_CALLS', conn_bi)

In [40]:
# Retrieve sales data

ventas_luz =  pd.read_sql("""
    SELECT con.contract_id, con.cups, tar.tariff_name as tariff, con.product_ekon_id as product, 
    con.sales_company_id, sls.sales_company_name, sls.channel_group, 
    'LIGHT' as business_type, date(co_start_date) as contract_start_date, esup.sp_zipcode as zipcode, 
    prov.province  FROM CON_ECONTRACT_DIM con
    left JOIN CON_TARIFF_DIM tar on con.tariff_ekon_id = tar.tariff_ekon_id 
    left join SLS_COMPANY_DIM sls on con.sales_company_id = sls.sales_company_id 
    left join CON_STATUS_DIM sta on con.co_status_id = sta.status_pk 
    left join CON_ESUPPLY_POINT_DIM esup on esup.contract_id = con.contract_id
    left join GEN_PROVINCE_DIM prov on left(esup.sp_zipcode, 2) = prov.province_nk
    left join OPE_SALES ope on con.contract_id = ope.contract_id
    WHERE registration_type IN('Cambio', 'Nueva Alta')
    and ope.contract_id is null
    and esup.sp_zipcode is not null
   
    """, conn_bi)

ventas_gas =  pd.read_sql("""
    SELECT con.contract_id, con.cups, tar.tariff_name as tariff, tar.tariff_name as product, 
    COALESCE(con.sales_company_id, 9997) AS sales_company_id, sls.sales_company_name, sls.channel_group, 
    'GAS' as business_type, date(co_start_date) as contract_start_date, esup.sp_zipcode as zipcode, 
    prov.province  FROM CON_GCONTRACT_DIM con
    left JOIN CON_TARIFF_DIM tar on con.tariff_id = tar.tariff_pk
    left join SLS_COMPANY_DIM sls on COALESCE(con.sales_company_id, 9997) = sls.sales_company_id 
    left join CON_STATUS_DIM sta on con.co_status_id = sta.status_pk 
    left join CON_GSUPPLY_POINT_DIM esup on esup.contract_id = con.contract_id
    left join GEN_PROVINCE_DIM prov on left(esup.sp_zipcode, 2) = prov.province_nk
    LEFT JOIN OPE_SALES ope on ope.contract_id = con.contract_id
    WHERE new_client_flag = 1
    and ope.contract_id is null
    and esup.sp_zipcode is not null
    
    """, conn_bi)

ventas = ventas_luz.append(ventas_gas)

insertNewDataToDB(ventas, 'OPE_SALES', conn_bi)

In [41]:
# Retrieve drop outs data

bajas_luz =  pd.read_sql("""
    SELECT con.contract_id, con.cups, tar.tariff_name as tariff, product_ekon_id as product, con.sales_company_id, 
    sls.sales_company_name, sls.channel_group, 'LIGHT' as business_type, 
    date(con.supply_end_date) as supply_end_date, esup.sp_zipcode as zipcode, prov.province, 
    lea.description  FROM CON_ECONTRACT_DIM con
    left JOIN CON_TARIFF_DIM tar on con.tariff_ekon_id = tar.tariff_ekon_id 
    left join SLS_COMPANY_DIM sls on con.sales_company_id = sls.sales_company_id 
    left join CON_ESUPPLY_POINT_DIM esup on esup.contract_id = con.contract_id
    left join GEN_PROVINCE_DIM prov on left(esup.sp_zipcode, 2) = prov.province_nk 
    left join CON_LEAVING_REASON_DIM lea on con.leaving_reason_id = lea.leaving_reason_id
    LEFT JOIN OPE_DROPS ope on ope.contract_id = con.contract_id
    where CUPS_out_of_Holaluz_flag = 1
    and con.co_status_id IN(9,10)
    and ope.contract_id is null
    and esup.sp_zipcode is not null
    """, conn_bi)

bajas_gas =  pd.read_sql("""
    SELECT con.contract_id, con.cups, tariff_name as tariff, tariff_name as product, 
    COALESCE(con.sales_company_id, 9997) AS sales_company_id, sls.sales_company_name, 
    sls.channel_group, 'GAS' as business_type, date(con.supply_end_date) as supply_end_date, 
    esup.sp_zipcode as zipcode, prov.province, co_state_detail as description FROM CON_GCONTRACT_DIM con
    left JOIN CON_TARIFF_DIM tar on con.tariff_id = tar.tariff_pk
    left join SLS_COMPANY_DIM sls on COALESCE(con.sales_company_id, 9997) = sls.sales_company_id
    left join CON_STATUS_DIM sta on con.co_status_id = sta.status_pk
    left join CON_GSUPPLY_POINT_DIM esup on esup.contract_id = con.contract_id
    left join GEN_PROVINCE_DIM prov on left(esup.sp_zipcode, 2) = prov.province_nk
    LEFT JOIN OPE_DROPS ope on ope.contract_id = con.contract_id
    where co_state_detail IN('terminated_new_sales_company', 'terminated_definitive', 'terminated_unpaid', 
    'terminated_withdrawal')
    and ope.contract_id is null
    and esup.sp_zipcode is not null
    
    """, conn_bi)

bajas = bajas_luz.append(bajas_gas)

insertNewDataToDB(bajas, 'OPE_DROPS', conn_bi)

In [42]:
# Retrieve leads data

leads_luz =  pd.read_sql("""
    select day, product, source, count(*) as num_leads from
    (select cups, product, day, source from
    (select cups, product, lead_id, date_trunc('day', created_at) as day
    from core.extended_leads where created_at > '2018-01-01') as a
    left join
    (select lead_id, source from lead.journey 
    where business_unit = 'electricity' and created_at > '2018-01-01') as b
    on a.lead_id = b.lead_id) as c
    group by day, product, source
    order by day, product, source
    """, conn_core)

leads_gas =  pd.read_sql("""
    select day, product, source, count(*) as num_leads from
    (select business_unit as product, source, date_trunc('day', created_at) as day from lead.journey 
    where business_unit = 'gas' and created_at > '2018-01-01') as a
    group by day, product, source
    order by day, product, source
    """, conn_core)

leads = leads_luz.append(leads_gas)

insertNewDataToDB(leads, 'OPE_LEADS', conn_bi)

In [46]:
already_in =  pd.read_sql(
        f"""
        SELECT * FROM OPE_LEADS
        """, conn_bi)
already_in.head()

,day,product,source,num_leads
0,2018-02-09,gas,web,1
1,2018-02-20,gas,web,1
2,2018-02-26,gas,web,1
3,2018-03-02,DOS Precios,web,1
4,2018-03-19,gas,web,1
